In [1]:
import cv2
import numpy as np

In [7]:
cap = cv2.VideoCapture('./Hand Video2.mov')
frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))

frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc('X','V','I','D')

out = cv2.VideoWriter("hand_output.avi", fourcc, 5.0, (1280,720))

ret, frame1 = cap.read()
ret, frame2 = cap.read()

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    kernel = np.ones((10, 10), np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=3)

    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(dilated)
    
    if cnt >= 2:
        min_x = 10000
        min_y = 10000 
        max_w = 0
        max_h = 0
        
        for i in range(1, cnt): 
            (x, y, w, h, area) = stats[i]

            # 노이즈 제거
            if area < 20:
                continue
                
            min_x = min(min_x, x)
            min_y = min(min_y, y)
            max_w = max(max_w, w)
            max_h = max(max_h, h)
      
        cv2.rectangle(frame1, (min_x, min_y, max_w, max_h), (0, 255, 255)) 
        cv2.putText(frame1,"(201711379) Motion detected!", (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.5,(0, 0,255))        
    

    image = cv2.resize(frame1, (1280,720))
    
    out.write(image)
    cv2.imshow("output", frame1)
    
    frame1 = frame2
    ret, frame2 = cap.read()
    
    if not ret:
        break

    if cv2.waitKey(20) == 27:
        break

cv2.destroyAllWindows()
cap.release()
out.release()